<a href="https://colab.research.google.com/github/larrymoreorless/Data-Mining/blob/master/subsetselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# activate R magic
%load_ext rpy2.ipython                                                                             

In [0]:
%%R
library(leaps)
library(caret)
library(nnet)
library(outliers)

set.seed(33)
train <- read.csv ("/Users/lawrenceanthony/Documents/Data Mining Methodology /DataSet/UCF Dataset 2018 - Training set.csv")

#omit missing variables.
train = na.omit(train)
sum(is.na(train))

#Remove variables
train$AppReceiveDate <- NULL
train$VehicleMake <- NULL
train$LTV <- NULL
train$CoMonthlyLiability <- NULL
train$CoMonthlyRent <-NULL
train$RequestType <- NULL
train$OccupancyStatus <- NULL
train$EmploymentStatus <- NULL

#Outlier Removal
#Remove_Outliers Function
remove_outliers <- function(x, na.rm = TRUE, ...) {
  qnt <- quantile(x, probs=c(.05, .95), na.rm = na.rm, ...)
  H <- 1.5 * IQR(x, na.rm = na.rm)
  y <- x
  y[x < (qnt[1] - H)] <- NA
  y[x > (qnt[2] + H)] <- NA
  y
}

summary(train$LoanNumber)


                  ------------<<<<<<<<<<< Outlier Removal >>>>>>>>----------
#No outliers for loan number
boxplot(train$LoanNumber)

#Do not use on credit score, removes 5k scores, which is highest predictor?
#ModifiedCreditScoreOR <- remove_outliers(train$ModifiedCreditScore)
#train$ModifiedCreditScore <- ModifiedCreditScoreOR

#1 NA!
ModifiedBankruptcyScoreOR <- remove_outliers(train$ModifiedBankruptcyScore)
train$ModifiedBankruptcyScore <- ModifiedBankruptcyScoreOR

EmployedMonthsOR <- remove_outliers(train$EmployedMonths)
train$EmployedMonths <- EmployedMonthsOR

#Removed 4654 obs, consider not using
#PrevEmployedMonthsOR <- remove_outliers(train$PrevEmployedMonths)
#train$PrevEmployedMonths <- PrevEmployedMonthsOR

#5269 observations
#CoEmployedMonthsOR <- remove_outliers(train$CoEmployedMonths)
#train$CoEmployedMonths <- CoEmployedMonthsOR

#5000 NAs, small range as well. consider not using
#CoPrevEmployedMonthsOR <- remove_outliers(train$CoPrevEmployedMonths)
#train$CoPrevEmployedMonths <- CoPrevEmployedMonthsOR

PrimeMonthlyIncomeOR <- remove_outliers(train$PrimeMonthlyIncome)
train$PrimeMonthlyIncome <- PrimeMonthlyIncomeOR

#Removed 3537 Observations, consider not using. 
#CototalMonthlyIncomeOR <- remove_outliers(train$CototalMonthlyIncome)
#train$CototalMonthlyIncome <- CototalMonthlyIncomeOR

#Removes 1772 incomes, consider not using?
#TotalMonthlyIncomeOR <- remove_outliers(train$TotalMonthlyIncome)
#train$TotalMonthlyIncome <- TotalMonthlyIncomeOR

PrimeMonthlyLiabilityOR <- remove_outliers(train$PrimeMonthlyLiability)
train$PrimeMonthlyLiability <- PrimeMonthlyLiabilityOR

PrimeMonthlyRentOR <- remove_outliers(train$PrimeMonthlyRent)
train$PrimeMonthlyRent <- PrimeMonthlyRentOR

TotalMonthlyDebtBeforeLoanOR <- remove_outliers(train$TotalMonthlyDebtBeforeLoan)
train$TotalMonthlyDebtBeforeLoan <- TotalMonthlyDebtBeforeLoanOR

VehicleYearOR <- remove_outliers(train$VehicleYear)
train$VehicleYear <- VehicleYearOR

VehicleMileageOR <- remove_outliers(train$VehicleMileage)
train$VehicleMileage <- VehicleMileageOR

TotalVehicleValueOR <- remove_outliers(train$TotalVehicleValue)
train$TotalVehicleValue <- TotalVehicleValueOR

AmountRequestedOR <- remove_outliers(train$AmountRequested)
train$AmountRequested <- AmountRequestedOR

#Consider not removing because it takes out 3066 observations
#DownPaymentOR <- remove_outliers(train$DownPayment)
#train$DownPayment <- DownPaymentOR

LoantermOR <- remove_outliers(train$Loanterm)
train$Loanterm <- LoantermOR

OccupancyDurationOR <- remove_outliers(train$OccupancyDuration)
train$OccupancyDuration <- OccupancyDurationOR

EstimatedMonthlyPaymentOR <- remove_outliers(train$EstimatedMonthlyPayment)
train$EstimatedMonthlyPayment <- EstimatedMonthlyPaymentOR

NumberOfOpenRevolvingAccountsOR <- remove_outliers(train$NumberOfOpenRevolvingAccounts)
train$NumberOfOpenRevolvingAccounts <- NumberOfOpenRevolvingAccountsOR

DTIOR <- remove_outliers(train$DTI)
train$DTI <- DTIOR


#                    ------------<<<<<<<<<<< Partition Data >>>>>>>>----------
  
#Sets the size of the data partition
sample <- sample.int(n = nrow(train), size = floor(.80*nrow(train)), replace = F)

#apply the size of the data partition to the data
train <- train[sample, ]
test  <- train[-sample, ]


#                    ------------<<<<<<<<<<< Run Subset Selection >>>>>>>>----------

#Run Subset Selection and View Results
ex <- regsubsets(LoanStatus ~., data = train, nvmax = 40, method = "exhaustive")
summary(ex)


#                    ------------<<<<<<<<<<<CHECKING NEW SUBSET SELECTION FOR BEST MODEL AFTER OUTLIER REMOVAL>>>>>>>>----------

#Begin with largest model, remove variables according to subset selection.

#27
check27 <- multinom(LoanStatus~ PrevEmployedMonths + OccupancyDuration + PrimeMonthlyIncome + PrimeMonthlyRent + EstimatedMonthlyPayment + CototalMonthlyIncome + DTI + CoPrevEmployedMonths + LoanNumber + DownPayment + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#26
check27 <- multinom(LoanStatus~ PrevEmployedMonths + OccupancyDuration + PrimeMonthlyIncome + EstimatedMonthlyPayment + CototalMonthlyIncome + DTI + CoPrevEmployedMonths + LoanNumber + DownPayment + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#25
check27 <- multinom(LoanStatus~ OccupancyDuration + PrimeMonthlyIncome + EstimatedMonthlyPayment + CototalMonthlyIncome + DTI + CoPrevEmployedMonths + LoanNumber + DownPayment + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#24
check27 <- multinom(LoanStatus~ OccupancyDuration + PrimeMonthlyIncome + EstimatedMonthlyPayment + TotalMonthlyIncome + DTI + CoPrevEmployedMonths + LoanNumber + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#23
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + TotalMonthlyIncome + DTI + CoPrevEmployedMonths + LoanNumber + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#22
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + CoPrevEmployedMonths + LoanNumber + CoEmployedMonths + TotalMonthlyDebtBeforeLoan + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)
#21
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + CoPrevEmployedMonths + LoanNumber + CoEmployedMonths + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#20
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + LoanNumber + CoEmployedMonths + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#19
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + LoanNumber + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear + Source, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#18 (Source, a two factor variable, is removed here. Correct Below)
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + LoanNumber + isNewVehicle + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#17
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + LoanNumber + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage + VehicleYear, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#16
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + LoanNumber + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#15
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + EstimatedMonthlyPayment + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#14
check27 <- multinom(LoanStatus~ EstimatedMonthlyPayment + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability + VehicleMileage, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#13
check27 <- multinom(LoanStatus~ EstimatedMonthlyPayment + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + EmployedMonths + AmountRequested + Loanterm + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#12
#XXXX This is other factor from Source, which was removed at line 18 XXXX

#11
check27 <- multinom(LoanStatus~ EstimatedMonthlyPayment + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + AmountRequested + Loanterm + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#10
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + AmountRequested + Loanterm + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#9
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + AmountRequested + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#8 Adds one, removes two.
check27 <- multinom(LoanStatus~ VehicleYear + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#7
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts + CoApplicantIndicator + PrimeMonthlyLiability, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#6 Add one, remove two
check27 <- multinom(LoanStatus~ PrimeMonthlyIncome + DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#5
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + ModifiedBankruptcyScore + MemberIndicator + NumberOfOpenRevolvingAccounts, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#4
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + MemberIndicator + NumberOfOpenRevolvingAccounts, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#3
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore + MemberIndicator, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#2
check27 <- multinom(LoanStatus~ DTI + ModifiedCreditScore, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)

#1
check27 <- multinom(LoanStatus~ ModifiedCreditScore, data = train, family = "binomial")
predicted27 <- predict(check27, test, type = 'class')
error27 <- table(predicted27, test$LoanStatus)
sum(diag(error27))/sum(error27)
1-sum(diag(error27))/sum(error27)




Error in (function (file = "", n = NULL, text = NULL, prompt = "?", keep.source = getOption("keep.source"),  : 
  <text>:37:31: unexpected input
36: 
37:                   ------------<<
                                  ^
